In [ ]:
"""
Student: Dinh Khac Tuyen
ID: 20214182
"""

from platform import python_version
import torch

print("python", python_version())
print("torch", torch.__version__)

python 3.7.12
torch 1.9.0+cu111


In [ ]:
!pip install -q datasets
from datasets import load_dataset
from pprint import pprint

squad_dataset = load_dataset('squad')
pprint(squad_dataset['validation'][0]) # 'context' contains the document

     |████████████████████████████████| 290 kB 31.1 MB/s 
     |████████████████████████████████| 243 kB 49.0 MB/s 
     |████████████████████████████████| 56 kB 5.7 MB/s 
     |████████████████████████████████| 1.3 MB 51.6 MB/s 
     |████████████████████████████████| 125 kB 58.4 MB/s 
     |████████████████████████████████| 160 kB 60.1 MB/s 
     |████████████████████████████████| 271 kB 46.8 MB/s 


Downloading:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/8.12M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset squad downloaded and prepared to /root/.cache/huggingface/datasets/squad/plain_text/1.0.0/d6ec3ceb99ca480ce37cdd35555d6cb2511d223b9150cce08a837ef62ffea453. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'answers': {'answer_start': [177, 177, 177],
             'text': ['Denver Broncos', 'Denver Broncos', 'Denver Broncos']},
 'context': 'Super Bowl 50 was an American football game to determine the '
            'champion of the National Football League (NFL) for the 2015 '
            'season. The American Football Conference (AFC) champion Denver '
            'Broncos defeated the National Football Conference (NFC) champion '
            'Carolina Panthers 24–10 to earn their third Super Bowl title. The '
            "game was played on February 7, 2016, at Levi's Stadium in the San "
            'Francisco Bay Area at Santa Clara, California. As this was the '
            '50th Super Bowl, the league emphasized the "golden anniversary" '
            'with various gold-themed initiatives, as well as temporarily '
            'suspending the tradition of naming each Super Bowl game with '
            'Roman numerals (under which the game would have been known as '
            '"Super

In [ ]:
print(len(squad_dataset['train']))
print(len(squad_dataset['validation']))

87599
10570


In [ ]:
import nltk
from nltk.corpus import stopwords
import numpy as np
import heapq
import re 
import math
class SimilaritySearch(object):
  def __init__(self,dataset):
    self.dataset=dataset
    nltk.download('stopwords')
    self.stop_words = set(stopwords.words('english'))
    self.doc_list=[]                  # list contain data_ids in the validation set that each data_id have different context to other data_id
                                      # len(doc_list)= the number of data in validation set that has different context
    self.create_doc_list()   
    self.vocab=[]                     # vocab: list of all word in train dataset (words that not stop word), or words in glove    
    self.data_dict={}                  # a dictionary of {"doc_id":{}}, where {} is the BoW, TFIDF , or BM25 dictionary (or glove embedding vector) of that document, for ex:{word1_idx:2,word2_idx:3,...}, the wordn_idx is index of word n in self.vocab
                                  
  def create_doc_list(self):
    # create list of docs from validation set
    # we have to ignore data_ids that share the same document in the dataset. Because data_ids that share same context are contiguous so we just need to check each['context'] with the previous 
    for i,each in enumerate(self.dataset['validation']):
      if i!=0: 
        if each['context'] == self.dataset['validation'][i-1]['context']:   # ignore 
          continue
      self.doc_list.append(each['id'])       
    print("Number of docs:{}".format(len(self.doc_list)))

  def preprocess(self,phrase):
      # handle abbreviation
      phrase = re.sub(r"won\'t", "will not", phrase)
      phrase = re.sub(r"can\'t", "can not", phrase)
      phrase = re.sub(r"n\'t", " not", phrase)
      phrase = re.sub(r"\'re", " are", phrase)
      phrase = re.sub(r"\'s", " is", phrase)
      phrase = re.sub(r"\'d", " would", phrase)
      phrase = re.sub(r"\'ll", " will", phrase)
      phrase = re.sub(r"\'t", " not", phrase)
      phrase = re.sub(r"\'ve", " have", phrase)
      phrase = re.sub(r"\'m", " am", phrase)
      # decapitalize
      phrase = phrase.lower()
      # remove unnecessary symbols
      symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
      for i in symbols:
        phrase = phrase.replace(i," ")
      return phrase

  def dict_normalize(self,dict):
    s=math.sqrt(sum([x*x for x in dict.values()]))
    for each in dict:
      dict[each]=dict[each]/s
    return dict

  # function to calculate inner product of 2 dict
  def dict_product(self,dict1,dict2):
    return sum(dict1[key]*dict2.get(key, 0) for key in dict1)
    
  # Perform multiply encode_input and data_dict[each] then find k most similar document to the query
  # if both the inputs are normalize, it becomes cosine similarity       
  def search(self, query: str, k: int) -> list:
    similarity={}
    encode_inp=self.encode_input(query)
    for each in self.data_dict:
      similarity[each]=self.dict_product(encode_inp,self.data_dict[each])    
    top_k_docs=heapq.nlargest(k,similarity,key=similarity.get)
    return top_k_docs


In [ ]:
"""
Problem 2.1: Create BoW 
"""

class BoW(SimilaritySearch):
  def __init__(self,dataset):
    super().__init__(dataset)
    self.create_vocab(self.dataset['train'])  # create vocab from train set
    self.createBoW(self.dataset)

  def create_vocab(self,dataset):
    # Create vocab from train dataset
    for i,each in enumerate(dataset):
      # First, we have to ignore data_ids that share the same document in the dataset. Because data_ids that share same context are contiguous so we just need to check each['context'] with the previous 
      if i!=0:    
        if each['context'] == dataset[i-1]['context']:   # ignore 
          continue
      
      doc=self.preprocess(each['context'])
      words=doc.split()
      for word in words:
        if word in self.stop_words:
          continue
        if word not in self.vocab:
            self.vocab.append(word)
    self.vocab_len=len(self.vocab)
    print("Number of word in vocab:{}".format(len(self.vocab)))


  def createBoW(self, dataset):  
    for i,each in enumerate(dataset['validation']):
      if each['id'] not in self.doc_list:
        continue
      doc=self.preprocess(each['context'])
      words=doc.split()
      doc_dict={}
      for word in words:
        if word not in self.vocab:
          continue
        word_idx=self.vocab.index(word)
        if word_idx in doc_dict:
          doc_dict[word_idx]+=1      # doc_dict saves the frequency of word in that doc
        else:
          doc_dict[word_idx]=1
      self.data_dict[each['id']]=doc_dict        
 


In [ ]:
"""
Problem 2.2: TF IDF Implementation
"""

import math
class TFIDF(BoW):
  def __init__(self,dataset):
    super().__init__(dataset)
    self.IDF=[]                               # array saves IDF value of words in vocab
    self.calculate_IDF()
    self.createTFIDF()
    
  def calculate_IDF(self):
    N=len(self.doc_list)                          # number of documents
    for word_idx,word in enumerate(self.vocab):
      doc_counter=0
      for doc in self.data_dict:
        if word_idx in self.data_dict[doc]:         # if doc contains the word
          doc_counter+=1
      if doc_counter:
        doc_counter=math.log(N/doc_counter)   
      self.IDF.append(doc_counter)

  def createTFIDF(self):
    # build TFTIDF data_dict from inheritted BoW data_dict, then save in self.data_dict
    for doc in self.data_dict:
      tfidf_dict={}
      doc_len=sum(self.data_dict[doc].values())            # len of doc in words 
      for word_idx,word in enumerate(self.vocab): 
        if word_idx in self.data_dict[doc]:    
          TF=self.data_dict[doc][word_idx]/doc_len
          tfidf_dict[word_idx]=TF*self.IDF[word_idx]
      self.data_dict[doc]=self.dict_normalize(tfidf_dict)
      
  # encode input dict based on TFIDF formular, same as docs
  def encode_input(self,query):
    sentence=self.preprocess(query)
    words=sentence.split()
    encode_inp={}
    #calculate word frequency vector of query
    for word in words:
      if word not in self.vocab:
        continue
      word_idx=self.vocab.index(word)
      if word_idx in encode_inp:
        encode_inp[word_idx]+=1
      else:
        encode_inp[word_idx]=1
    query_len=sum(encode_inp.values())
    #calculate TFIDF vector of query
    for word_idx in encode_inp:        
      TF=encode_inp[word_idx]/query_len
      encode_inp[word_idx]=TF*self.IDF[word_idx]    
    return self.dict_normalize(encode_inp)


In [ ]:
import time

def test_1_search(retrieval,t):
  # Test time of 1 search
  print("Document comes with question:")
  print(squad_dataset['validation'][t]['context'])
  query=squad_dataset['validation'][t]['question']
  print("Question:"+query)
  start=time.time()
  list_docs=tfidf.search(query,10)
  end=time.time()
  print("Search time:{:.6f}s".format(end-start))
  print("Searched result:")
  found =-1
  for i in range(10):
    print("Document {}:".format(i+1))
    found_data=next(each for each in squad_dataset['validation'] if each["id"] == list_docs[i])
    print(found_data['context'])
    if found_data['context']==squad_dataset['validation'][t]['context']:
      found=i
  print()
  if found>=0:
    print("Correct search, Document{}".format(found+1))
  else:
    print("Wrong search")

"""
Function to calculate recall score, which is calculate from the number of success search (result documents contain the correct document)
devided by the number of total search 
I test questions belong to data that has id in retrival.doc_list
"""
def recall_score(dataset,retrieval,k):
  success_search=0
  for i,data in enumerate(dataset['validation']):
    if data['id'] not in retrieval.doc_list:
      continue
    list_docs_searched=retrieval.search(data['question'],k)
    if data['id'] in list_docs_searched:
      success_search+=1
  return success_search/len(retrieval.doc_list)

In [ ]:
tfidf=TFIDF(squad_dataset)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Number of docs:2067
Number of word in vocab:107107


In [ ]:
test_1_search(tfidf,22)

Document comes with question:
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Question:What team was the AFC champion?
Search time:0.016762s
Searched result:
Document 1:
Super Bowl 50 was an American football game to determine the champion of the National Football League 

In [ ]:
"""
Problem 2.3: TF-IDF Recall Score
"""
print("TF-IDF Recall Score:{}".format(recall_score(squad_dataset,tfidf,10)))

TF-IDF Recall Score:0.8393807450411224


In [ ]:
"""
Problem 2.4: BM25 implementation and recall score
"""

class BM25(BoW):
  def __init__(self,dataset):
    super().__init__(dataset)
    self.IDF=[]                               # array saves IDF value of words
    self.calculate_IDF()
    self.createBM25()
    
  def calculate_IDF(self):
    N=len(self.doc_list)                          # number of documents
    for word_idx,word in enumerate(self.vocab):
      doc_counter=0
      for doc in self.data_dict:
        if word_idx in self.data_dict[doc]:         # if doc contains the word
          doc_counter+=1
      if doc_counter:
        doc_counter=math.log(N/doc_counter)   
      self.IDF.append(doc_counter)
 
  # Create BM25 dict, which saves BM25 value of each word corresponding to each doc. 
  def createBM25(self):
    k=2
    b=0.75
    totallen=0
    for doc in self.data_dict:
      totallen+=sum(self.data_dict[doc].values())
    avg_doclen=totallen/len(self.doc_list)

    for doc in self.data_dict:
      bm25_dict={}
      doc_len=sum(self.data_dict[doc].values())            # len of doc 
      for word_idx,word in enumerate(self.vocab): 
        if word_idx in self.data_dict[doc]:       
          freq=self.data_dict[doc][word_idx]
          bm25_dict[word_idx]=self.IDF[word_idx]*freq*(k+1)/(freq+k*(1-b+b*doc_len/avg_doclen))
      self.data_dict[doc]=self.dict_normalize(bm25_dict)

  # encode_input(): encode the query to a dict:{word1:n1,word2:n2,...} is the frequency of word in the query
  # in BM25 formular, the score of a query toward a doc is the sum of BM25 value of (each_word_in_query, doc), so is basically the inner product of word frequency dict of query and doc dict
  # then, we can use the inheritted search function
  def encode_input(self,query):
    sentence=self.preprocess(query)
    words=sentence.split()
    encode_inp={}
    #calculate word frequency dict of query
    for word in words:
      if word not in self.vocab:
        continue
      word_idx=self.vocab.index(word)
      if word_idx in encode_inp:
        encode_inp[word_idx]+=1
      else:
        encode_inp[word_idx]=1  
    return self.dict_normalize(encode_inp)



In [ ]:
bm25=BM25(squad_dataset)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of docs:2067
Number of word in vocab:107107


In [ ]:
test_1_search(bm25,22)

Document comes with question:
Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.
Question:What team was the AFC champion?
Search time:0.019168s
Searched result:
Document 1:
Super Bowl 50 was an American football game to determine the champion of the National Football League 

In [ ]:
print("BM25 Recall Score:{}".format(recall_score(squad_dataset,bm25,10)))

BM25 Recall Score:0.8819545234639574


In [ ]:
"""
Problem 3.1: Glove Average Search Implementation
"""

import torch.nn as nn
class GloveAverage(SimilaritySearch):
  def __init__(self,dataset):
    super().__init__(dataset)
    self.word2idx={}
    self.create_vocab_and_embedding()
    self.create_embedding_docs()

  def create_vocab_and_embedding(self):
    idx = 0
    vectors = []
    with open('/content/drive/MyDrive/pytorch_data/glove.6B/glove.6B.50d.txt', 'rb') as f:
      for l in f:
        line = l.decode().split()
        word = line[0]
        self.vocab.append(word)
        self.word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

    glove = {w: vectors[self.word2idx[w]] for w in self.vocab}
    print("Number of word in vocab: {}".format(len(self.vocab)))
    # init a matrix contain weight from glove for embedding layer 
    matrix_len = len(self.vocab)
    weights_matrix = np.zeros((matrix_len, 50))
    #for each word in vocab, get the vector of word from glove
    for i, word in enumerate(self.vocab):
        weights_matrix[i] = glove[word]
    self.embedding = nn.Embedding(matrix_len, 50)
    self.embedding.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    self.embedding.weight.requires_grad = False 

  def create_embedding_docs(self):
    for each in self.dataset['validation']:
      if each['id'] not in self.doc_list:
        continue
      doc=self.preprocess(each['context'])
      words=doc.split()
      doc_vec=[]
      for word in words:
        if word in self.stop_words:
          continue
        if word not in self.vocab:
          continue
        doc_vec.append(self.word2idx[word]) 
      doc_glove=self.embedding(torch.LongTensor(doc_vec))    # (len(doc),embed_dims)
      doc_v=torch.mean(doc_glove,0)       # compute doc_vec, which is the mean of vec of all word
      doc_v=doc_v.numpy()
      doc_v=doc_v/np.linalg.norm(doc_v)
      self.data_dict[each['id']]=doc_v 

  def encode_input(self,query):
    query=self.preprocess(query)
    words=query.split()
    query_vec=[]
    for word in words:
      if word in self.stop_words:
        continue
      if word not in self.vocab:
        continue
      query_vec.append(self.word2idx[word]) 
    query_glove=self.embedding(torch.LongTensor(query_vec))         # (len(doc),embed_dims)
    query_v=torch.mean(query_glove,0)       # compute query_vec, which is the mean of vec of all word
    query_v=query_v.numpy()
    query_v=query_v/np.linalg.norm(query_v)
    return query_v  


  def search(self, query: str, k: int) -> list:
    similarity={}
    encode_inp=self.encode_input(query)
    for each in self.data_dict:
      similarity[each]=np.dot(encode_inp,self.data_dict[each])
    top_k_docs=heapq.nlargest(k,similarity,key=similarity.get)
    return top_k_docs



In [ ]:
glove_average=GloveAverage(squad_dataset)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of docs:2067
Number of word in vocab: 400000


In [ ]:
test_1_search(glove_average,222)

Document comes with question:
The Panthers offense, which led the NFL in scoring (500 points), was loaded with talent, boasting six Pro Bowl selections. Pro Bowl quarterback Cam Newton had one of his best seasons, throwing for 3,837 yards and rushing for 636, while recording a career-high and league-leading 45 total touchdowns (35 passing, 10 rushing), a career-low 10 interceptions, and a career-best quarterback rating of 99.4. Newton's leading receivers were tight end Greg Olsen, who caught a career-high 77 passes for 1,104 yards and seven touchdowns, and wide receiver Ted Ginn, Jr., who caught 44 passes for 739 yards and 10 touchdowns; Ginn also rushed for 60 yards and returned 27 punts for 277 yards. Other key receivers included veteran Jerricho Cotchery (39 receptions for 485 yards), rookie Devin Funchess (31 receptions for 473 yards and five touchdowns), and second-year receiver Corey Brown (31 receptions for 447 yards). The Panthers backfield featured Pro Bowl running back Jonath

In [ ]:
"""
Problem 3.2: Glove average Recall score
"""

print("Glove Average Recall Score:{}".format(recall_score(squad_dataset,glove_average,10)))

Glove Average Recall Score:0.6168359941944848


In [ ]:
!pip install faiss-cpu

     |████████████████████████████████| 8.4 MB 15.4 MB/s 


In [ ]:
"""
Problem 3.3: Glove Average using Faiss and recall score
"""

import faiss
import torch.nn as nn
class GloveAverageFaiss(SimilaritySearch):
  def __init__(self,dataset):
    super().__init__(dataset)
    self.word2idx={}
    self.create_vocab_and_embedding()
    self.create_embedding_docs()
    self.search_database=[]             # 2d matrix build from self.data_dict
    self.idx_to_docid={}                # mapping row index in search_database matrix to doc_id
    self.create_search_database()

  def create_vocab_and_embedding(self):
    idx = 0
    vectors = []
    with open('/content/drive/MyDrive/pytorch_data/glove.6B/glove.6B.50d.txt', 'rb') as f:
      for l in f:
        line = l.decode().split()
        word = line[0]
        self.vocab.append(word)
        self.word2idx[word] = idx
        idx += 1
        vect = np.array(line[1:]).astype(np.float)
        vectors.append(vect)

    glove = {w: vectors[self.word2idx[w]] for w in self.vocab}
    print(len(self.vocab))
    # init a matrix contain weight from glove for embedding layer 
    matrix_len = len(self.vocab)
    weights_matrix = np.zeros((matrix_len, 50))
    #for each word in vocab, get the vector of word from glove
    for i, word in enumerate(self.vocab):
        weights_matrix[i] = glove[word]
    self.embedding = nn.Embedding(matrix_len, 50)
    self.embedding.load_state_dict({'weight': torch.from_numpy(weights_matrix)})
    self.embedding.weight.requires_grad = False 

  def create_embedding_docs(self):
    for each in self.dataset['validation']:
      if each['id'] not in self.doc_list:
        continue
      doc=self.preprocess(each['context'])
      words=doc.split()
      doc_vec=[]
      for word in words:
        if word in self.stop_words:
          continue
        if word not in self.vocab:
          continue
        doc_vec.append(self.word2idx[word]) 
      doc_glove=self.embedding(torch.LongTensor(doc_vec))    # (len(doc),embed_dims)
      doc_v=torch.mean(doc_glove,0)       # compute mean of vec of all word
      doc_v=doc_v.numpy()
      doc_v=doc_v/np.linalg.norm(doc_v)
      self.data_dict[each['id']]=doc_v 

  def encode_input(self,query):
    query=self.preprocess(query)
    words=query.split()
    query_vec=[]
    for word in words:
      if word in self.stop_words:
        continue
      if word not in self.vocab:
        continue
      query_vec.append(self.word2idx[word])
    query_glove=self.embedding(torch.LongTensor(query_vec))         # (len(doc),embed_dims)
    query_v=torch.mean(query_glove,0)       # compute mean of vec of all word
    query_v=query_v.numpy()
    query_v=query_v/np.linalg.norm(query_v)
    return query_v  

  def create_search_database(self):
    idx=0
    for each in self.data_dict:
      self.idx_to_docid[idx]=each
      self.search_database.append(self.data_dict[each])
      idx+=1
    self.search_database=np.array(self.search_database).astype('float32')
    # Build search index from search_database
    self.index = faiss.IndexFlatIP(self.search_database.shape[1])
    self.index.add(self.search_database)

  def search(self, query: str, k: int) -> list:
    top_k_docs=[]
    encode_inp=self.encode_input(query)
    if len(encode_inp):     # input not empty
      encode_inp=np.expand_dims(encode_inp, axis=0)
      encode_inp.astype('float32')
      D, I = self.index.search(encode_inp, k)
      for idx in np.squeeze(I):
        if idx==-1:
          continue
        top_k_docs.append(self.idx_to_docid[idx])
    return top_k_docs

In [ ]:
ga_faiss=GloveAverageFaiss(squad_dataset)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Number of docs:2067
400000


In [ ]:
test_1_search(ga_faiss,32)

Document comes with question:
The Panthers finished the regular season with a 15–1 record, and quarterback Cam Newton was named the NFL Most Valuable Player (MVP). They defeated the Arizona Cardinals 49–15 in the NFC Championship Game and advanced to their second Super Bowl appearance since the franchise was founded in 1995. The Broncos finished the regular season with a 12–4 record, and denied the New England Patriots a chance to defend their title from Super Bowl XLIX by defeating them 20–18 in the AFC Championship Game. They joined the Patriots, Dallas Cowboys, and Pittsburgh Steelers as one of four teams that have made eight appearances in the Super Bowl.
Question:What year was the Carolina Panthers franchise founded?
Search time:0.021740s
Searched result:
Document 1:
Despite waiving longtime running back DeAngelo Williams and losing top wide receiver Kelvin Benjamin to a torn ACL in the preseason, the Carolina Panthers had their best regular season in franchise history, becoming t

In [ ]:

print("GloveAverageFaiss Recall Score:{}".format(recall_score(squad_dataset,ga_faiss,10)))


GloveAverageFaiss Recall Score:0.6168359941944848


In [ ]:
"""
Problem 3.4: Compare search time between Glove Average using native operator and faiss
"""

def test_1000_search_time(retrieval,dataset):
  time_taken=0
  for i,each in enumerate(dataset['validation']):
    if i>999:
      break
    start=time.time()
    list_docs_searched=retrieval.search(each['question'],10)
    end=time.time()
    time_taken+=end-start
  return time_taken/1000

print("Time taken for GloveAverage:{:.6f}".format(test_1000_search_time(glove_average,squad_dataset)))
print("Time taken for GloveAverageFaiss:{:.6f}".format(test_1000_search_time(ga_faiss,squad_dataset)))

Time taken for GloveAverage:0.005329
Time taken for GloveAverageFaiss:0.001989
